In [1]:
print("Ok")

Ok


In [2]:
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader
import pinecone
from langchain.prompts import PromptTemplate

/home/codespace/.python/current/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Create pinecone cluster, get API key

In [3]:
PINECONE_API_KEY = "37d70112-c425-4c0b-80bd-846d1429eaf9"
PINECONE_API_ENV = ""

Extract pdf data

In [4]:

from langchain.document_loaders import PyPDFLoader
def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob = "*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [6]:
extracted_data = load_pdf("data/")

FileNotFoundError: Directory not found: 'data/'

In [7]:
# extracted_data

Create text chunks

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
print("Number of ch chunks is", len(text_chunks))

Download embedding model

In [8]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings = download_hugging_face_embeddings()

/home/codespace/.python/current/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/codespace/.python/current/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

Test embedding model

In [11]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


Vector representation of "Hello World"

In [12]:
query_result

[-0.03447727859020233,
 0.03102324716746807,
 0.006734943948686123,
 0.02610895037651062,
 -0.03936202824115753,
 -0.1603025197982788,
 0.06692401319742203,
 -0.006441460456699133,
 -0.047450434416532516,
 0.014758843928575516,
 0.07087527215480804,
 0.05552758276462555,
 0.019193317741155624,
 -0.026251306757330894,
 -0.010109581984579563,
 -0.026940494775772095,
 0.022307422012090683,
 -0.022226620465517044,
 -0.1496926248073578,
 -0.017493054270744324,
 0.0076762172393500805,
 0.0543522872030735,
 0.00325441500172019,
 0.03172592446208,
 -0.08462139964103699,
 -0.02940598502755165,
 0.05159560963511467,
 0.048124004155397415,
 -0.00331481802277267,
 -0.05827918276190758,
 0.041969262063503265,
 0.02221064455807209,
 0.128188818693161,
 -0.022338906303048134,
 -0.011656313203275204,
 0.06292835623025894,
 -0.032876331359148026,
 -0.09122605621814728,
 -0.031175334006547928,
 0.052699558436870575,
 0.04703483358025551,
 -0.08420306444168091,
 -0.030056189745664597,
 -0.020744858309626

Initializing Pinecone

In [13]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "mnnit-chatbot"

Creating embeddings for each text chunk and storing in the vector database

In [14]:
from langchain_pinecone import PineconeVectorStore
import os
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
docsearch = PineconeVectorStore.from_documents(
    text_chunks,
    embeddings,
    index_name = index_name
)

NameError: name 'text_chunks' is not defined

Trying out the index

In [ ]:
# # docsearch = Pinecone.from_existing_index(index_name, embeddings)
query = "What is the NIRF rank of MNNIT?"
# query_vector = embeddings.embed_query(query)
index = pc.Index(index_name)
# results = index.query(vector = query_vector, top_k = 3, include_values = True)
# #docs = docsearch.similarity_search(query, k = 3)
# print(results)


from langchain_pinecone import PineconeVectorStore
vectorstore = PineconeVectorStore(index=index, embedding=embeddings)

results = vectorstore.similarity_search(
    query,
    k=3  # return 3 most relevant docs 
)

print(results)

Load llm model

In [ ]:
import langchain
import langchain_community
from langchain_community.llms import CTransformers
llm = CTransformers(model = "model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type = "llama",
                    verbose = True,
                    config = {'max_new_tokens':512,
                              'temperature':0.8})

Entering LLM Prompt Template

In [ ]:
print(llm)

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Don't answer any queries not related to MNNIT.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
from langchain.chains import RetrievalQA
qa=RetrievalQA.from_chain_type(
    llm = llm,
    chain_type="stuff",
    retriever = vectorstore.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query":user_input})
    print("Response: ",result["result"])